In [6]:
import json
import pandas as pd
import os

schedulers = ['CFS', 'FIFO', 'RR', 'RM']
column_names = []
exp_result_dir = os.path.join(os.getcwd(), '../exp_results')

data = {}

# Load data from JSON files
for scheduler in schedulers:
    file_path = os.path.join(exp_result_dir, f'{scheduler}_result.json')
    with open(file_path, 'r') as file:
        results = json.load(file)
        data[scheduler] = results

# Extract column names
column_names.extend(["Task", "DeadLine(us)", "WCET(us)"])
column_names.extend([f"{scheduler}_Sched" for scheduler in schedulers])
column_names.extend([f"{scheduler}_WCRT(us)" for scheduler in schedulers])
column_names.extend([f"{scheduler}_AVG(us)" for scheduler in schedulers])

In [7]:
# Initialize an empty list to store data
data_rows = []

def find_task_info_by_name(task_name, scheduler):
    for info in data[scheduler]:
        if info['task_name'] == task_name:
            return info

# Fill in the list with task information
for task in data[schedulers[0]]:
    row_data = {}
    task_name = task['task_name']
    row_data['Task'] = task_name
    row_data['DeadLine(us)'] = int(task['deadline_ns'] / 1000)
    row_data['WCET(us)'] = int(task['wcet_ns'] / 1000)
    for scheduler in schedulers:
        task_info = find_task_info_by_name(task_name, scheduler)
        row_data[f"{scheduler}_WCRT(us)"] = int(task_info['wcrt_ns'] / 1000)
        row_data[f"{scheduler}_AVG(us)"] = int(task_info['avg_response_time_ns'] / 1000)
        if task_info['wcrt_ns'] < task_info['deadline_ns']:
            row_data[f"{scheduler}_Sched"] = "Yes"
        else:
            row_data[f"{scheduler}_Sched"] = "No"
    data_rows.append(row_data)

# Create DataFrame from the list of data
analysis_result = pd.DataFrame(data_rows, columns=column_names)

In [8]:
analysis_result

,Task,DeadLine(us),WCET(us),CFS_Sched,FIFO_Sched,RR_Sched,RM_Sched,CFS_WCRT(us),FIFO_WCRT(us),RR_WCRT(us),RM_WCRT(us),CFS_AVG(us),FIFO_AVG(us),RR_AVG(us),RM_AVG(us)
0,Task_200ms,200000,139,Yes,Yes,Yes,Yes,17964,4053,3410,338,585,534,554,150
1,Task_10ms,10000,14628,No,No,No,No,46549,34671,34460,60609,12092,11605,11535,11665
2,Angle_Sync,6660,2614,No,No,No,No,28194,18325,22929,52066,2605,2492,2448,2072
3,Task_20ms,20000,10415,No,No,No,No,54506,36040,34388,56113,15449,12232,12143,11013
4,Task_50ms,50000,731,Yes,Yes,Yes,Yes,24867,16871,16138,11551,710,1760,1616,650
5,ISR_9,6000,74,No,No,No,No,21708,14646,17430,47254,62,113,65,77
6,ISR_8,1700,57,No,No,No,No,22145,15957,19274,48926,62,117,70,77
7,ISR_7,4900,67,No,No,No,No,22582,16483,19347,44826,70,125,78,88
8,Task_5ms,5000,463,No,No,No,No,25366,16939,21716,65082,328,904,966,3025
9,ISR_2,9500,5,No,Yes,No,No,20934,4366,18939,47778,12,153,122,2406


In [14]:
# cwd = os.getcwd()
# print(cwd)
# assert False
import numpy as np

tasks_info_file = "../../../dataset/FMTV_application/generated_FMTV_json/tasks_info.json"
tasks_info = json.load(open(tasks_info_file, 'r'))

column_names = ["name", "period", "wcet", "read", "execution", "write"]

data_rows = []
for task in tasks_info:
    if task["isPeriodic"] and (task["period_ns"] > 20000000) :
        row_data = {}
        row_data["name"] = task["task_name"]
        row_data["period"] = int(task["period_ns"] / 1000)
        # row_data["core"] = task["core_index"]
        row_data["read"] = int(task["phased_read"] / 1000)
        row_data["write"] = int(task["phased_write"] / 1000)
        phased_execution = task["phased_execution"]
        phased_execution = np.array(phased_execution) / 1000
        row_data["execution"] = np.max(phased_execution).astype(int)
        row_data["wcet"] = row_data["read"] + row_data["execution"] + row_data["write"]
        data_rows.append(row_data)
        
tasks_data = pd.DataFrame(data_rows, columns=column_names)
tasks_data
    

,name,period,wcet,read,execution,write
0,Task_200ms,200000,141,24,98,19
1,Task_50ms,50000,685,32,633,20
2,Task_100ms,100000,11327,174,11036,117
3,Task_1000ms,1000000,88,32,23,33
4,Non_RT_Task_1000ms_core0,1000000,100000,0,100000,0
5,Non_RT_Task_1000ms_core1,1000000,100000,0,100000,0
6,Non_RT_Task_1000ms_core2,1000000,100000,0,100000,0
7,Non_RT_Task_1000ms_core3,1000000,100000,0,100000,0


In [10]:
tasks_info[0].keys()

dict_keys(['task_name', 'core_index', 'priority', 'time_ns', 'isRTTask', 'isPeriodic', 'period_ns', 'phased_read', 'phased_write', 'phased_execution'])